# Spotting deputies

## Preliminaries

In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


#This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
import re

# from collections import defaultdict
import pandas as pd
# import numpy as np
# #import xmltodict
# import networkx as nx
# from sqlalchemy import text


repo_dir = os.path.split(os.getcwd())[0]

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

import sys
!{sys.executable} -m pip install -U numpy

Requirement already up-to-date: numpy in /Users/rikhoekstra/.local/share/virtualenvs/republic-project-Av06XlOZ/lib/python3.8/site-packages (1.21.0)


In [2]:
%reload_ext numpy


In [3]:
abbr_delegates = pd.read_parquet("/Users/rikhoekstra/develop/republic_clean/republic/data/csvs/abbreviated_delegates.parquet") 

In [4]:
# let's repress the warnings, esp. from pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [68]:
import run_attendancelist

In [69]:
from republic.data.delegate_database import abbreviated_delegates, found_delegates


In [70]:
def overl(x, y):
    try:
        if x.overlaps(y):
            return True
        else:
            return False
    except:
        print(x, type(x))
        raise
abbreviated_delegates.loc[abbreviated_delegates.p_interval.apply(lambda x: overl(x, pd.Interval(1623, 1677)))]

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval.left,p_interval.right,h_life.left,h_life.right,p_interval,h_life
1041,7050,de Bie,None,1628-12-31,NaN,Rekenkamer Holland,ontvanger-generaal,"PeriodIndex(['1584-01-01', '1584-01-02', '1584...",False,False,1584,1628,1550,1628,"(1584, 1628]","(1550, 1628]"
1047,7120,van Oranje,1567-01-01,1625-12-31,NaN,"Hof van Holland, Zeeland en West-Friesland (14...","raadsheer, stadhouder","PeriodIndex(['1585-11-01', '1585-11-02', '1585...",False,False,1585,1625,1567,1625,"(1585, 1625]","(1567, 1625]"
1062,10783,de Bie,1550-01-01,None,NaN,Raad van State,thesaurier-generaal,"PeriodIndex(['1586-01-01', '1586-01-02', '1586...",False,False,1586,1628,1550,1650,"(1586, 1628]","(1550, 1650]"
1084,12343,van Aylva,1635-01-01,None,NaN,"Admiraliteit in Friesland (1596-1795), Gedeput...","gecommitteerde, lid","PeriodIndex(['1663-03-21', '1663-03-22', '1663...",False,False,1663,1677,1635,1699,"(1663, 1677]","(1635, 1699]"
1085,12344,van Aylva,1644-01-01,1705-01-31,NaN,"Admiraliteit in Friesland (1596-1795), Gedeput...","gecommitteerde, lid","PeriodIndex(['1672-03-16', '1672-03-17', '1672...",False,False,1672,1701,1644,1705,"(1672, 1701]","(1644, 1705]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7935,19992,Singendonck,None,1669-12-31,NaN,Gedeputeerde Staten van het Kwartier Nijmegen ...,"ordinaris gedeputeerde, gedeputeerde","PeriodIndex(['1621-07-27', '1621-07-28', '1621...",True,True,1621,1669,1577,1669,"(1621, 1669]","(1577, 1669]"
7941,20000,van Wijnbergen,None,1703-12-31,NaN,Staten-Generaal der Verenigde Nederlanden (158...,"ordinaris gedeputeerde, lid","PeriodIndex(['1660-10-15', '1660-10-16', '1660...",True,True,1660,1705,1616,1703,"(1660, 1705]","(1616, 1703]"
7942,20001,van Arnhem tot Harselo,1636-01-01,1716-12-12,NaN,"Ridderschap van het Kwartier Veluwe, Gedeputee...","lid, gedeputeerde, extraordinaris raadsheer","PeriodIndex(['1658-01-01', '1658-01-02', '1658...",False,False,1658,1716,1636,1716,"(1658, 1716]","(1636, 1716]"
7943,20002,van Arnhem,1596-01-01,1649-09-06,NaN,"Ridderschap van het Kwartier Veluwe, Gedeputee...","lid, extraordinaris raadsheer, raadsheer","PeriodIndex(['1619-01-01', '1619-01-02', '1619...",False,False,1619,1649,1596,1649,"(1619, 1649]","(1596, 1649]"


In [322]:
runner = run_attendancelist.RunAll(year=1730, outdir='/Users/rikhoekstra/republic_out')


logging to /Users/rikhoekstra/republic_out/attendancelist.log


In [323]:
runner.year

1730

In [284]:
def get_president(ob, txt):
    ofset = 0  # in case we find no president marker
    end = 0
    prez = ''
    prae = ''
    matches = ps.find_matches(text=txt)
    presidents = [m for m in matches if m.label == 'president']
    president = best_match(presidents)
    print (president)
    if president:
        ofset = getattr(president, "offset") or 0
        prez = getattr(president, "string") or ''
    presentibi = [m for m in matches if m.label == 'presentibus']
    if presentibi:
        presentibus = best_match(presentibi)
        prae = presentibus.string or ''
        end = getattr(presentibus, "offset") or len(txt)
    spns = {}
    if ofset != 0:
        preamble_span = (0, ofset - 1)
        preamble = txt[preamble_span[0]:preamble_span[1]]
        spns['preamble'] = (preamble, preamble_span)
    pre_span = (ofset, ofset + len(prez))
    spns['pre'] = (prez, pre_span)
    presentibus_span = (end, end + len(prae))
    spns['presentibus'] = (prae, presentibus_span)
    searchpat = pat % (re.escape(prez), re.escape(prae))
    print(searchpat)
    r = re.search(searchpat, txt)
    print(r)
    if r and r.group(1):
        heer = r.group(1).strip()
        heer = re.sub('[^\s\w]*', '', heer)
        #logging.info(heer)
        s1 = txt.find(heer)
        s2 = s1 + len(heer)
        span = (s1, s2)  # get_span_from_regex(r)
        spns['president'] = (heer, span)
        for kw in spns.keys():
            s = spns[kw][0]
            spn = spns[kw][1]
            ob.set_span(span=spn, pattern=s, clas=kw)
            setattr(ob, 'found', {'president': heer})
            # if prae != '':
            #     prespan = get_span_from_regex(prae)
            #     mt.set_span(prespan, "presentibus")
        return heer

In [287]:
import random
from republic.analyser.attendance_lists.pattern_finders import president_searcher, get_president
#president_searcher(runner.searchobs)
ps = make_president_searcher()
ks = random.choices(list(runner.searchobs.keys()), k=20)
pres = []
prs = []
for T in ks:
    ob = runner.searchobs[T].matched_text
    txt = ob.item
    res = ps.find_matches(text=txt)

    presidents = [m for m in res if m.label=='president']
    pibus = [m for m in res if m.label=='presentibus']
    if presidents:
        president = best_match(presidents)
        pres.append(president)
        print('president: ', president.string)
    else:
        print('no president', T)
    if pibus:
        prbus = best_match(pibus)
        prs.append(prbus)
        print('pibus: ', prbus.string)
    else:
         print('no pibus', T)
    try:
        print('resultaat: ', gp(president, prbus)[1])
        print('getprres:', get_president(ob, r"%s(.*)%s", ps, txt))
    except (UnboundLocalError, IndexError):
        pass
    


president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  de Kempenaar
getprres: de Kempenaar
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  uan Glinstra
getprres: uan Glinstra
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  de Milan Visconti
getprres: de Milan Visconti
no president session-1730-05-01-num-1
no pibus session-1730-05-01-num-1
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  van Lynden
getprres: van Lynden
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  van Lynden
getprres: van Lynden
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  van Wassenaer
getprres: van Wassenaer
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  van Blocklandt
getprres: van Blocklandt
president:  PRAESIDE, Den Hecre
no pibus session-1730-04-07-num-1
president:  PRAESIDE, Den Heere
pibus:  PRAESENTIBUS
resultaat:  vander Steen
getprres: vander Steen
president:  DRAEYVDE, Ben Heere
pibus:  PRAESE

In [267]:
presidents

[PhraseMatch(phrase: "PRAESIDE Den Heere", variant: "PRAESIDE Den Heere",string: "PROAESFDY, Den Heere", offset: 217)]

In [244]:
import re
def gp(president, praesentibus):
    ofset = getattr(presidents[0], "offset") or 0
    prez = getattr(presidents[0], "string") or ''
    if pibus:
        presentibus = best_match(pibus)
        prae = presentibus.string or ''
        end = getattr(presentibus, "offset") or len(txt)
    searchpat = pat % (re.escape(prez), re.escape(prae))
    r = re.search(searchpat, txt)
    heer = r.group(1).strip()
    rheer = re.sub('[^\s\w]*', '', heer)
    rex
    return(heer, rheer)
#print(ofset, prez, prae, end)

In [324]:
runner.initial_find()

1. finding presidents
293 found
2.find provincial extraordinaris gedeputeerden


In [302]:
from republic.analyser.attendance_lists.pattern_finders import province_searcher
%time province_searcher(searchobs)

CPU times: user 16min 5s, sys: 13.6 s, total: 16min 19s
Wall time: 16min 34s


In [304]:
from republic.analyser.attendance_lists.pattern_finders import province_searcher
%time province_searcher(searchobs)

CPU times: user 16min 25s, sys: 14.1 s, total: 16min 39s
Wall time: 16min 54s


In [327]:
from republic.analyser.attendance_lists.pattern_finders import make_province_searcher, base_config
prs = make_province_searcher(config=base_config)
for T in ks:
    ob = runner.searchobs[T].matched_text
    txt = ob.item
    print(prs.find_matches(txt, include_variants=True))

[PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Gelderlandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van Gelderlandt",string: "met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt", offset: 123), PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Zeelandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van Zeelandt",string: "een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt", offset: 127), PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Zeelandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van Zeelandt",string: "Een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt", offset: 353), PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Groningen", variant: "met twee extraordinaris gedeputeerde uyt de provincie van Groningen",string: "Een extraordinaris Gedeputeerde uyt de Provi

[PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Gelderlandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van Gelderlandt",string: "met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt", offset: 124), PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Zeelandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van Zeelandt",string: "een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt", offset: 128), PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Hollandt ende West-Vrieslandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van Hollandt ende West-Vrieslandt",string: "een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Wassenaer, van", offset: 128)]
[PhraseMatch(phrase: "met een extraordinaris gedeputeerde uyt de provincie van Gelderlandt", variant: "met een extraordinaris gedeputeerde uyt de provincie van

In [326]:
searchobs = runner.searchobs
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


In [321]:
ob.serialize()

'<span style="color:rgb(178,223,138)">Veeneris den 30. Junii 1730.</span> <span style="color:rgb(31,120,180)">PRAESIDE,Den Heere</span> <span style="color:rgb(166,206,227)">Queisen</span> .  <span style="color:rgb(51,160,44)">PRAESENTIBUS</span> , De Heeren van Lynden, van Welderen, Coulman, van Eck tot Nergena, me <span style="color:rgb(251,154,153)">t twee extraordinaris Gedeputeerden uyt de Provincie van Gelderland</span> t. Van Wassenaer, Eelbo, vander Dussen, vander Marck, Raadtpensionaris van Slin <span style="color:rgb(251,154,153)">gelandt</span> . Noey,  <span style="color:rgb(251,154,153)">met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt</span> . Vander Waayen , Sleterdyck. Van Tamminga.PE Resouuen gisteren, genomen, '

In [311]:
ob.get_fragments()

[{'type': 'preamble',
  'idnr': 0,
  'pattern': 'Veeneris den 30. Junii 1730.',
  'begin': 0,
  'end': 28,
  'delegate_id': 0,
  'delegate_name': '',
  'delegate_gewest': 0,
  'delegate_score': 0},
 {'type': 'pre',
  'idnr': 0,
  'pattern': 'PRAESIDE,Den Heere',
  'begin': 29,
  'end': 47,
  'delegate_id': 0,
  'delegate_name': '',
  'delegate_gewest': 0,
  'delegate_score': 0},
 {'type': 'president',
  'idnr': 0,
  'pattern': 'Queisen',
  'begin': 48,
  'end': 55,
  'delegate_id': 0,
  'delegate_name': '',
  'delegate_gewest': 0,
  'delegate_score': 0},
 {'type': 'unmarked', 'pattern': '. ', 'end': 55, 'begin': 56},
 {'type': 'presentibus',
  'idnr': 0,
  'pattern': 'PRAESENTIBUS',
  'begin': 57,
  'end': 69,
  'delegate_id': 0,
  'delegate_name': '',
  'delegate_gewest': 0,
  'delegate_score': 0},
 {'type': 'unmarked',
  'pattern': ', De Heeren van Lynden, van Welderen, Coulman, van Eck tot Nergena, me',
  'end': 69,
  'begin': 138},
 {'type': 'province',
  'idnr': 0,
  'pattern': 't

In [313]:
fragments = ob.get_fragments()
# fragment += "[{}]".format(i.idnr) # for now
outfragments = []
for fragment in fragments:
    if fragment['type'] != 'unmarked':
        tcolor = ob.colormap.get(fragment['type']) or 'unknown'
        ff = ob.color_match(fragment['pattern'], color=tcolor)
        outfragments.append(ff)
    else:
        outfragments.append(fragment['pattern'])
txt = " ".join(outfragments)  # may want to turn this in object property
print(txt)

<span style="color:rgb(178,223,138)">Veeneris den 30. Junii 1730.</span> <span style="color:rgb(31,120,180)">PRAESIDE,Den Heere</span> <span style="color:rgb(166,206,227)">Queisen</span> .  <span style="color:rgb(51,160,44)">PRAESENTIBUS</span> , De Heeren van Lynden, van Welderen, Coulman, van Eck tot Nergena, me <span style="color:rgb(251,154,153)">t twee extraordinaris Gedeputeerden uyt de Provincie van Gelderland</span> t. Van Wassenaer, Eelbo, vander Dussen, vander Marck, Raadtpensionaris van Slin <span style="color:rgb(251,154,153)">gelandt</span> . Noey,  <span style="color:rgb(251,154,153)">met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt</span> 


In [316]:
f = ob.get_fragments()
f[-2:]

[{'type': 'province',
  'idnr': 0,
  'pattern': 'met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt',
  'begin': 300,
  'end': 365,
  'delegate_id': 0,
  'delegate_name': '',
  'delegate_gewest': 0,
  'delegate_score': 0},
 {'type': 'unmarked', 'pattern': '', 'begin': 365, 'end': 439}]

In [310]:
htmlout=[]
for T in ks:
    ob = searchobs[T].matched_text
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
HTML("<br><hr><br>".join(htmlout))

In [325]:
%time runner.gather_found_delegates()


3. finding unmarked text
4. joining presidents and delegates
total 1 found
CPU times: user 13.4 s, sys: 255 ms, total: 13.6 s
Wall time: 14 s


In [208]:
ekwz['PRASENTIBUS']

['Prasentibus',
 'PRESENTIBUS',
 'TIBÚS',
 'TIBDUS',
 'TIEBUS',
 'PRA&ASEHNTIBUS',
 'TIBÚUS',
 'TIDUS',
 'TLBUS',
 'TIBUS',
 'TIBÜS',
 'TIBDUDS',
 'PR&SENTIBUS',
 'ENTIBUS',
 'PRASELNTIBUS']

In [204]:
runner.presidents

['ESIDE Den Heere van Wassenaer',
 'ESIDDE Den Heere van Wassenaer PRAEZS',
 'ESIDE Den Heere van WY assenaer',
 'ESIDE Den Heere van Wassenaer',
 'ESIDE Den Heere van Wassenaer',
 'ESIDE Den Heere van Borsselen',
 'van Borselen',
 'van Borsselen',
 'ESIDE Den Heere van Borsselen',
 'ESITDE Den Heere van Borsselen',
 'ESTDE Deu Heere van Borsselen',
 'ESIDE Den Heere Jaats van Amerongen',
 'ESIDE Den Heere Zaats van Amerongen',
 'ESIDE Den Heere Zaais van Amerongen',
 'ESIDE Den Heere Zaats van Amerongen PRAEA',
 'van Renswoude',
 'ESIDE Deu Heere van Renswoude PRAZS',
 'ESITDE Den Heere van Glinstra',
 'ESIDE Den Heere uan Glinstra',
 'ESIDE Den Heere van Glinstra',
 'ESIDE Den Heere van Glinstra',
 'ESTDE Deu Heere van Glinstra',
 'ESIDE Den Heere van Rechteren',
 'ESIDE Den Heere van Rechteren',
 'ESIDE Den Heere van Rechteren',
 'ESIDE Den Heere van Rechteren PRAEZES',
 'ESIDE Den Heere van Rechteren',
 'van Rechteren',
 'ESIDE Den Heere van Tamminga',
 'van Tamminga',
 'van Tammin

In [199]:
txt = "PRAESIDEDen Heere van Lynden PRAESENWIDBDU 8 De Heeren van Welderen van Singendonck Rasch Torck van Esk tot Norgena met twee extraordinana Gedepuieerden uyt de Provincie van Gelderlandt Van Weasscnaer le Leu de Wilbem vander Marck vander Ramborst Raadtpensionaris van Siingelandt Van Citters van Borsselen de Jongh Bout Noey Hungrosgne vun Hoorn Verelst met een entraordinaris Gedeputeerde uyt de Provincie wan Zeelandt Van Rensqwoude Sloterdyck met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt Bueisen Van Berchuys  Clant,"
pat = "%s(.*)%s"  # pat = "%s.*(.*)%s" 

In [195]:
from republic.analyser.attendance_lists.pattern_finders import make_president_searcher
ps = make_president_searcher()
matches = ps.find_matches(text=txt, use_word_boundaries=False, include_variants=True)
presidents = [m for m in matches if m.label=='president']

president = best_match(presidents)
ofset = getattr(president, "offset")
prez = getattr(president, "string")
presentibi = [m for m in matches if m.label == 'presentibus']
if presentibi:
    presentibus = best_match(presentibi)
    prae = presentibus.string or ''
    end = getattr(presentibus, "offset") or len(txt)
spns = {}
if ofset != 0:
    preamble_span = (0, ofset - 1)
    preamble = txt[preamble_span[0]:preamble_span[1]]
    spns['preamble'] = (preamble, preamble_span)
pre_span = (ofset, ofset + len(prez))
spns['pre'] = (prez, pre_span)
presentibus_span = (end, end + len(prae))
spns['presentibus'] = (prae, presentibus_span)
searchpat = pat % (re.escape(prez), re.escape(prae))
r = re.search(searchpat, txt)
heer = r.group(1).strip()
heer

'van Lynden'

In [194]:
prae

'PRAESEN'

In [190]:
mn = min([abs(len(m.phrase.exact_string) - len( m.string)) for m in presidents])

In [191]:
candidates = [m for m in presidents if abs(len(m.phrase.exact_string) - len( m.string)) == mn]
candidate = max([m for m in candidates if score_match(m) == mx], key=lambda x: score_match(x))
candidate

ValueError: max() arg is an empty sequence

In [121]:
runner.verify_matches()

5. verifying matches
verifying spans
updating merged delegates database
6. finished to attendance lists and found database


In [ ]:
def best_match(matches=[]):
    """
    calculates best match from fuzzysearch matches based on scorematch. Returns single match
    TODO: this does not account for best length if match scores are equal"""
    if matches:
        mx = max([score_match(m) for m in matches])
        candidates = [m for m in matches if score_match(m) == mx]
        candidate = max(candidates, key=lambda x: len(x.string))
        return candidate

In [ ]:
from republic.analyser.attendance

In [10]:
from fuzzy_search.fuzzy_context_searcher import FuzzyContextSearcher
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_searcher import FuzzySearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

In [11]:
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fcs = FuzzyContextSearcher(config=config)
fps = FuzzyPhraseSearcher(config=config)

phrase = PhraseModel()

In [20]:
from republic.data.delegate_database import ekwz
# create a list of domain keywords and phrases
# variants = [{'phrase': k, 'variants': v} for k, v in ekwz.items()]
# # create a PhraseModel object from the domain phrases
# phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fps.index_phrase_model(phrase_model=phrase_model)
fcs.index_phrase_model(phrase_model=phrase_model)

In [104]:
vs = ['PRASIDE Den Heere',
          'PRESIDE Den Heere',
          'P R A E S I D E Den Heere',
          'PRAESIDE Den Heere',
          'P R A S I D E Den Heere',
          'P R E S I D E Den Heere',
          'P R AE S I D E Den Heere',
          'DR AS 1D E Den Heere',
          'PR ASL DE Den Heere',
          'PR A31 DE; Den Heere',
          'BR JE 3.1. DE, Den Heere']
vs = vs + ekwz['PRAS']
pvs = ['PRASENTIBUS',
                'PRESENTIBUS',
                'P R A E S E N T I B U S',
                'P RAE SE N TI B U S',
                'PRA&SENTIBUS']
variants = [{'phrase': 'PRAESIDE Den Heere', 'label':"president", 'variants': vs},
            {'phrase':'PRAESENTIBUS',  "variants":ekwz['PRASENTIBUS']}]
phrase_model = PhraseModel(model=variants)
phrase_model.json()

[{'phrase': {'phrase': 'PRAESIDE Den Heere',
   'label': 'president',
   'variants': ['PRASIDE Den Heere',
    'PRESIDE Den Heere',
    'P R A E S I D E Den Heere',
    'PRAESIDE Den Heere',
    'P R A S I D E Den Heere',
    'P R E S I D E Den Heere',
    'P R AE S I D E Den Heere',
    'DR AS 1D E Den Heere',
    'PR ASL DE Den Heere',
    'PR A31 DE; Den Heere',
    'BR JE 3.1. DE, Den Heere',
    'PRAES',
    'PR&S',
    'PREAS',
    'Pros',
    'PRES',
    'PR&AS',
    'PRA&S',
    'PRA',
    'BRAS']},
  'variants': ['B', 'R', 'A', 'S'],
  'label': ['president']},
 {'phrase': {'phrase': 'PRAESENTIBUS',
   'variants': ['Prasentibus',
    'PRESENTIBUS',
    'TIBÚS',
    'TIBDUS',
    'TIEBUS',
    'PRA&ASEHNTIBUS',
    'TIBÚUS',
    'TIDUS',
    'TLBUS',
    'TIBUS',
    'TIBÜS',
    'TIBDUDS',
    'PR&SENTIBUS',
    'ENTIBUS',
    'PRASELNTIBUS']},
  'variants': ['P', 'R', 'A', 'S', 'E', 'L', 'N', 'T', 'I', 'B', 'U', 'S']}]

In [14]:
p = run_attendancelist.ekwz['PRASENTIBUS']
pvs = ['PRASENTIBUS',
                'PRESENTIBUS',
                'P R A E S E N T I B U S',
                'P RAE SE N TI B U S',
                'PRA&SENTIBUS']
p+pvs

NameError: name 'run_attendancelist' is not defined

In [15]:
from fuzzy_search.fuzzy_string import score_levenshtein_similarity_ratio

In [22]:
res = fps.find_matches(text="PRAESIDE Den Heere, PRAESENTIBUS, PRESN", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
# for item in res:
#     print(item.score_levenshtein_similarity())
from scipy.special import softmax
from numpy import argmax
def score_match(match):
    result = sum([match.levenshtein_similarity, match.character_overlap, match.ngram_overlap, score_levenshtein_similarity_ratio(str(match.variant.exact_string),str(match.phrase.exact_string))])
    return result

b = argmax(softmax([score_match(i) for i in res]))
res[b]

PhraseMatch(phrase: "PRAESIDE Den Heere", variant: "PRAESIDE Den Heere",string: "PRAESIDE Den Heere", offset: 0)

In [17]:
score_levenshtein_similarity_ratio(str(i.variant.exact_string),str(i.phrase.exact_string))

NameError: name 'i' is not defined

In [24]:
res = fps.find_matches(text="PRAESIDE Den Heere, PRAESENTIBUS, PRESN", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
mx = max([score_match(m) for m in res])
[m for m in res if score_match(m)==mx]
r = res[0]
r.end - r.offset
ranges = [set(range(r.offset, r.end)) for r in res]
p = run_attendancelist.pairwise
setpair = []
for pair in p(ranges):
    pair = list(pair)
    pair.sort()
    if pair[0].intersection(y):
        if pair[0].issubset(pair[1]):
            setpair.append(pair[1])
    

NameError: name 'run_attendancelist' is not defined

In [30]:
matches=res
mx = max([score_match(m) for m in matches])
candidates = [m for m in matches if score_match(m)==mx]
candidate = max(candidates, key=lambda x: len(x.string))
candidate

PhraseMatch(phrase: "PRAESIDE Den Heere", variant: "PRAESIDE Den Heere",string: "PRAESIDE Den Heere", offset: 0)

In [218]:
from itertools import combinations
set([x for x in combinations(ranges, 2) if set(x[0]) & set(x[1]) ])


{(range(0, 4), range(0, 4)),
 (range(12, 17), range(12, 16)),
 (range(12, 17), range(12, 17)),
 (range(12, 17), range(12, 18)),
 (range(12, 18), range(12, 16)),
 (range(12, 18), range(12, 17)),
 (range(12, 18), range(12, 18))}

In [190]:
ranges
x = [{12, 13, 14, 15, 16},
 {12, 13, 14, 15, 16, 17}]
x.sort()
if x[0].issubset(x[1]):
    print(x[1],x)

{12, 13, 14, 15, 16} [{12, 13, 14, 15, 16}, {12, 13, 14, 15, 16, 17}]


In [197]:
print(ranges, max(ranges, key=lambda x: len(x)))

[{0, 1, 2, 3}, {0, 1, 2, 3}, {12, 13, 14, 15, 16}, {12, 13, 14, 15, 16, 17}, {12, 13, 14, 15, 16}, {12, 13, 14, 15, 16, 17}, {12, 13, 14, 15}] {12, 13, 14, 15, 16, 17}


In [201]:
from heapq import merge


def find_overlap_and_non_overlap(ranges):
    starts, ends = zip(*ranges)
    overlap = range(max(starts), min(ends)+1)
    if not overlap:
        return ([], range(min(starts), max(ends)+1), [])

    less_non_overlap = range(*heapq.nsmallest(2, starts))
    end, start = heapq.nlargest(2, ends)
    greater_non_overlap = range(start+1, end+1)
    return (overlap, less_non_overlap, greater_non_overlap)


In [202]:
ranges = [range(r.offset, r.end) for r in res]
find_overlap_and_non_overlap(ranges)

ValueError: too many values to unpack (expected 2)

In [93]:
item.score_levenshtein_similarity()

1.0

In [94]:
# create a list of domain keywords and phrases
domain_phrases = [
    # terms for the chair and attendants of a meeting
    "PRAESIDE",
    "PRAESENTIBUS",
    # some weekdays in Latin
    "Veneris", 
    "Mercuri",
    # some date phrase where any date in January 1725 should match
    "den .. Januarii 1725"
]

# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(phrases=domain_phrases)

# register the phrase model with the searcher
fps.index_phrase_model(phrase_model)

# take some example texts: meetings of the Dutch States General in January 1725
text1 = "ie Veucris den 5. Januaris 1725. PR&ASIDE, Den Heere Bentinck. PRASENTIEBUS, De Heeren Jan Welderen , van Dam, Torck , met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Maasdam , vanden Boeizelaar , Raadtpenfionaris van Hoornbeeck , met een extraordinaris Gedeputeerde uyt de Provincie van Hollandt ende Welt-Vrieslandt. Velters, Ockere , Noey; van Hoorn , met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van Renswoude , van Voor{t. Van Schwartzenbergh, vander Waayen, Vegilin Van I{elmuden. Van Iddekinge ‚ van Tamminga."

text2 = "Mercuri: den 10. Jangarii , | 1725. ia PRESIDE, Den Heere an Iddekinge. PRA&SENTIBUS, De Heeren /an Welderen , van Dam, van Wynbergen, Torck, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderland. Van Maasdam , Raadtpenfionaris van Hoorn=beeck. Velters, Ockerfe, Noey. Taats van Amerongen, van Renswoude. Vander Waasen , Vegilin, ’ Bentinck, van I(elmaden. Van Tamminga."

In [128]:
t1 = runner.searchobs[T].matched_text.item
fps.find_matches(text=t1, include_variants=True, use_word_boundaries=False, allow_overlapping_matches=False)

[PhraseMatch(phrase: "anden", variant: "anden",string: "a den", offset: 3),
 PhraseMatch(phrase: " een ", variant: " een ",string: " den ", offset: 4),
 PhraseMatch(phrase: "den", variant: "den",string: "den", offset: 5),
 PhraseMatch(phrase: "den .. Januarii 1725", variant: "den .. Januarii 1725",string: "den 2. Januarii 17", offset: 5),
 PhraseMatch(phrase: "January", variant: "January",string: "Januar", offset: 12),
 PhraseMatch(phrase: "Januarii", variant: "Januarii",string: "Januarii", offset: 12),
 PhraseMatch(phrase: "urii", variant: "urii",string: "uarii", offset: 15),
 PhraseMatch(phrase: "rii", variant: "rii",string: "rii", offset: 17),
 PhraseMatch(phrase: "PRAESIDE Den Heere", variant: "PRAES",string: "PRAES", offset: 26),
 PhraseMatch(phrase: "PRAESIDE Den Heere", variant: "PRA",string: "PRA", offset: 26),
 PhraseMatch(phrase: " een ", variant: " een ",string: " Den ", offset: 35),
 PhraseMatch(phrase: "Den", variant: "Den",string: "Den", offset: 36),
 PhraseMatch(phrase: 

In [109]:
runner.junksweeper

In [108]:
umt


[['Quiut', 'Quint', '&uint', 'Vuint', 'Suint'],
 ['Amerongen', "a'merongen"],
 ['Yaayen',
  'Wiaayen',
  'YWaayen',
  'Waauyen',
  'Waayen',
  'Weaayen',
  "H'aayen",
  'Haayen',
  "HW'aayen",
  'Woaayen',
  'aayen',
  'Wauayen',
  'Haaayen',
  "W'aayen",
  'HWiaayen',
  "YW'aaayen",
  "B'aayen",
  "Y'aayen"],
 ['Heassenaer',
  'Wassenaer',
  'Weasscnaer',
  'Wiassenaer',
  'Weassenaer',
  "B'assenaer",
  'Yassenaer',
  "H'assenaer",
  'assenaer',
  'Hassenaer',
  'Wassenser',
  'HWiassenaer',
  'Wassenaar'],
 ['Borssêlen',
  'Bôrsselen',
  'Borselen',
  'Bursselen',
  'Borsselen',
  'Beorsselen'],
 ['Berchuys', 'Berghuys', 'Berchuss'],
 ['ber', 'Beur'],
 ['Blocklandt',
  'ocklandt',
  'Bilocklandt',
  'Blocklandi',
  'Blocklandr',
  'Blocklandit'],
 ['Bout', 'Boat', 'Buut'],
 ['Bolthe',
  'Holthen',
  'Houithe',
  'HBolthe',
  'HolThe',
  'Holten',
  'Holthe',
  'Hoithe',
  'Hosthe'],
 ['Horn', 'Boorn', 'Born', 'borst', 'Bors', 'Boru', 'Hoorn'],
 ['Brincke', 'Brinck'],
 ['Bruyninv', '

In [106]:
t1 = runner.searchobs[T].matched_text.item
fcs.find_matches(text=t1,include_variants=True, use_word_boundaries=False, allow_overlapping_matches=False)

[PhraseMatchInContext(phrase: "Den", variant: "den",string: "den", offset: 5), context: "Luna den 2. Januarii 1730.PRAESIDE, Den Heere van Wassenaer.De Heeren van Lynden, van Wiynbergen, BEE VV DES",
 PhraseMatchInContext(phrase: "january", variant: "Januarii",string: "Januarii", offset: 12), context: "Luna den 2. Januarii 1730.PRAESIDE, Den Heere van Wassenaer.De Heeren van Lynden, van Wiynbergen, BEE VV DESREXVAXNDEDEO",
 PhraseMatchInContext(phrase: "urii", variant: "rii",string: "rii", offset: 17), context: "Luna den 2. Januarii 1730.PRAESIDE, Den Heere van Wassenaer.De Heeren van Lynden, van Wiynbergen, BEE VV DESREXVAXNDEDEO",
 PhraseMatchInContext(phrase: "PRAS", variant: "PRAES",string: "PRAES", offset: 26), context: "Luna den 2. Januarii 1730.PRAESIDE, Den Heere van Wassenaer.De Heeren van Lynden, van Wiynbergen, BEE VV DESREXVAXNDEDEODNDEEBIEENE",
 PhraseMatchInContext(phrase: "PRAS", variant: "PRA",string: "PRA", offset: 26), context: "Luna den 2. Januarii 1730.PRAESIDE, Den

In [ ]:
for phrase_dict in variants:
            main_phrase = as_phrase_object(phrase_dict, ngram_size=self.ngram_size, skip_size=self.skip_size)
            if main_phrase.phrase_string not in self.phrase_index:
                if add_new_phrases:
                    self.add_phrase(main_phrase)
                else:
                    continue
            if "variants" not in main_phrase.metadata:
                continue
            for variant_phrase_string in main_phrase.metadata["variants"]:
                variant_phrase = as_phrase_object(variant_phrase_string, ngram_size=self.ngram_size,
                                                  skip_size=self.skip_size)
                self.add_variant(variant_phrase, main_phrase)

In [87]:
umt = runner.find_unmarked_text()
out = []
for s in umt:
    out.append(' '.join(s))
allout = ' '.join(out)
type(allout)

3. finding unmarked text


str

In [119]:


# unmarked_text = ''.join(umt)
# splittekst = re.split(pattern="\s", string=unmarked_text)
# for s in splitteks
res = fuzzy_searcher.find_matches(text="we are all PRAESENTIBUS", include_variants=False, use_word_boundaries=False)
r = max(res, key=lambda x: x.score_levenshtein_similarity())
#fks.find_candidates(s, include_variants=True)

In [121]:
res

[PhraseMatch(phrase: "PRAESENTIBUS", variant: "PRAESENTIBUS",string: "PRAESENTIBUS", offset: 11)]

In [122]:
searchob = runner.searchobs['session-1730-02-14-num-1']
unmarked_text = ''.join(searchob.matched_text.get_unmatched_text())
print(unmarked_text)
splittekst = re.split(pattern="\s", string=unmarked_text)
for s in splittekst:
    if len(s) > 2 and len(fuzzy_searcher.find_matches(text=s, include_variants=False, use_word_boundaries=False)) == 0:
        print('ok')

, De Heeren vander Steen. Van Wassenaer, van Blocklandt, vanden Born, Bors van Waveren, vande Poll, van Cattenborgh. Van Borsselen, Bout, Noey, van Hoorn. Van Renswoude. Van Glinstra. Van Rechteren, van Haarsolte, van Isselmuden. Van Tamminga.DO Ntosutien gisteren, 2enomen, zyn gelesen en geresumeert, gelijck oock geresumeert ende gearresteert zyn de Depesches daar uyt resulteerende.IS ter Vergaderinge getoont ende geexhibeert een Lyste van de MagistraatsPersoonen ende Offieianten der Steede ende Lande van Willemstadt, welcke den eedt hebben gepraesteert, in gevolge haar Hoogh Mogende Resolutie van den twintighsten December seventien hondert vyftien. DD WAAE op geen resolutie is gevallen.
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [81]:
runner.junksweeper

In [80]:
matchsearch = runner.identify_delegates()
for T in runner.searchobs:
    searchob = runner.searchobs[T]
    mo = run_attendancelist.MatchAndSpan(searchob.matched_text, junksweeper=runner.junksweeper, 
                                         previously_matched=runner.found_delegates, 
                                         match_search=matchsearch)

Empty DataFrame
Columns: [ref_id, geboortejaar, sterfjaar, colleges, functions, period, active_stgen, was_gedeputeerde, variants, name, found_in, id, sg, period_active.left, period_active.right, hypothetical_life.left, hypothetical_life.right, p_interval.left, p_interval.right]
Index: []


AttributeError: 'FuzzyKeywordSearcher' object has no attribute 'find_matches'

In [201]:
from IPython.display import HTML
htmlout=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"\n<h4>{T}</h4>\n" + rest
    if url:
        rest += f"""<br/><br/><a href='{url}'>link naar {T}-image</a><br/>"""
    htmlout.append(rest)
#out.reverse()
HTML("<br><hr><br>".join(htmlout))


NameError: name 'searchobs' is not defined

In [30]:
import statistics
from collections import Counter
txtlens = [len(runner.searchobs[T].text) for T in runner.searchobs]
print(min(txtlens), max(txtlens))
print(statistics.mean(txtlens))
print(statistics.stdev(txtlens))
print(Counter(txtlens).most_common())

0 2548
524.9498327759197
304.563832960001
[(0, 6), (513, 5), (430, 4), (476, 3), (484, 3), (500, 3), (425, 3), (485, 3), (469, 3), (494, 3), (541, 3), (446, 3), (550, 3), (454, 3), (364, 3), (488, 3), (389, 3), (367, 2), (560, 2), (750, 2), (572, 2), (563, 2), (465, 2), (543, 2), (506, 2), (542, 2), (483, 2), (558, 2), (456, 2), (570, 2), (538, 2), (487, 2), (514, 2), (390, 2), (416, 2), (530, 2), (439, 2), (509, 2), (573, 2), (418, 2), (458, 2), (311, 2), (419, 2), (403, 2), (379, 2), (377, 2), (473, 2), (337, 2), (426, 2), (428, 2), (391, 2), (502, 2), (356, 2), (557, 1), (527, 1), (642, 1), (655, 1), (675, 1), (605, 1), (611, 1), (693, 1), (680, 1), (673, 1), (636, 1), (594, 1), (635, 1), (666, 1), (612, 1), (589, 1), (2505, 1), (584, 1), (625, 1), (630, 1), (614, 1), (606, 1), (622, 1), (452, 1), (769, 1), (745, 1), (539, 1), (440, 1), (564, 1), (492, 1), (529, 1), (518, 1), (511, 1), (525, 1), (547, 1), (627, 1), (533, 1), (2024, 1), (540, 1), (522, 1), (646, 1), (575, 1), (580, 1

In [56]:
fuzzysearch_config = {
    "char_match_threshold": 0.7,
    "ngram_threshold": 0.5,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
from republic.analyser.attendance_lists.parse_delegates import make_junksweeper
from republic.data.delegate_database import ekwz
from republic.model.republic_phrase_model import month_names_early, month_names_late
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
provincies = ['Holland', 'Zeeland', 'West-Vriesland', 'Gelderland', 'Overijssel', 'Utrecht', 'Friesland']
months = month_names_early + month_names_late
junksweeper = FuzzyKeywordSearcher(config=fuzzysearch_config)
indexkeywords = list(ekwz.keys()) + months + provincies
junksweeper.index_keywords(indexkeywords)
#ekwz = indexkeywords
fks = make_junksweeper(ekwz)


In [107]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel

# highger matching thresholds for higher quality OCR/HTR (higher precision, recall should be good anyway)
# lower matching thresholds for lower quality OCR/HTR (higher recall, as that's the main problem)
config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.8,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

# initialize a new searcher instance with the config
fuzzy_searcher = FuzzyPhraseSearcher(config)

# create a list of domain keywords and phrases
variants = [{'phrase': k, 'variants': v} for k, v in ekwz.items()]
# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(model=variants,)
#phrase_model.add_phrases(ekwz.keys())
# phrase_model.add_variants(variants)
fuzzy_searcher.index_phrase_model(phrase_model=phrase_model)

In [135]:
res = fuzzy_searcher.find_matches(text="Heer. dikke Heeren", include_variants=True, use_word_boundaries=False, allow_overlapping_matches=True)
for item in res:
    print(item.score_levenshtein_similarity())

0.8
1.0
0.8
0.8333333333333334
1.0
1.0
1.0


In [134]:
item.score_levenshtein_similarity()

0.8

In [111]:
# create a list of domain keywords and phrases
domain_phrases = [
    # terms for the chair and attendants of a meeting
    "PRAESIDE",
    "PRAESENTIBUS",
    # some weekdays in Latin
    "Veneris", 
    "Mercuri",
    # some date phrase where any date in January 1725 should match
    "den .. Januarii 1725"
]

# create a PhraseModel object from the domain phrases
phrase_model = PhraseModel(phrases=domain_phrases)

# register the phrase model with the searcher
fuzzy_searcher.index_phrase_model(phrase_model)

# take some example texts: meetings of the Dutch States General in January 1725
text1 = "ie Veucris den 5. Januaris 1725. PR&ASIDE, Den Heere Bentinck. PRASENTIEBUS, De Heeren Jan Welderen , van Dam, Torck , met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Maasdam , vanden Boeizelaar , Raadtpenfionaris van Hoornbeeck , met een extraordinaris Gedeputeerde uyt de Provincie van Hollandt ende Welt-Vrieslandt. Velters, Ockere , Noey; van Hoorn , met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van Renswoude , van Voor{t. Van Schwartzenbergh, vander Waayen, Vegilin Van I{elmuden. Van Iddekinge ‚ van Tamminga."

text2 = "Mercuri: den 10. Jangarii , | 1725. ia PRESIDE, Den Heere an Iddekinge. PRA&SENTIBUS, De Heeren /an Welderen , van Dam, van Wynbergen, Torck, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderland. Van Maasdam , Raadtpenfionaris van Hoorn=beeck. Velters, Ockerfe, Noey. Taats van Amerongen, van Renswoude. Vander Waasen , Vegilin, ’ Bentinck, van I(elmaden. Van Tamminga."

In [ ]:
for phrase_dict in variants:
            main_phrase = as_phrase_object(phrase_dict, ngram_size=self.ngram_size, skip_size=self.skip_size)
            if main_phrase.phrase_string not in self.phrase_index:
                if add_new_phrases:
                    self.add_phrase(main_phrase)
                else:
                    continue
            if "variants" not in main_phrase.metadata:
                continue
            for variant_phrase_string in main_phrase.metadata["variants"]:
                variant_phrase = as_phrase_object(variant_phrase_string, ngram_size=self.ngram_size,
                                                  skip_size=self.skip_size)
                self.add_variant(variant_phrase, main_phrase)

In [87]:
umt = runner.find_unmarked_text()
out = []
for s in umt:
    out.append(' '.join(s))
allout = ' '.join(out)
type(allout)

3. finding unmarked text


str

In [119]:


# unmarked_text = ''.join(umt)
# splittekst = re.split(pattern="\s", string=unmarked_text)
# for s in splitteks
res = fuzzy_searcher.find_matches(text="we are all PRAESENTIBUS", include_variants=False, use_word_boundaries=False)
r = max(res, key=lambda x: x.score_levenshtein_similarity())
#fks.find_candidates(s, include_variants=True)

In [121]:
res

[PhraseMatch(phrase: "PRAESENTIBUS", variant: "PRAESENTIBUS",string: "PRAESENTIBUS", offset: 11)]

In [122]:
searchob = runner.searchobs['session-1730-02-14-num-1']
unmarked_text = ''.join(searchob.matched_text.get_unmatched_text())
print(unmarked_text)
splittekst = re.split(pattern="\s", string=unmarked_text)
for s in splittekst:
    if len(s) > 2 and len(fuzzy_searcher.find_matches(text=s, include_variants=False, use_word_boundaries=False)) == 0:
        print('ok')

, De Heeren vander Steen. Van Wassenaer, van Blocklandt, vanden Born, Bors van Waveren, vande Poll, van Cattenborgh. Van Borsselen, Bout, Noey, van Hoorn. Van Renswoude. Van Glinstra. Van Rechteren, van Haarsolte, van Isselmuden. Van Tamminga.DO Ntosutien gisteren, 2enomen, zyn gelesen en geresumeert, gelijck oock geresumeert ende gearresteert zyn de Depesches daar uyt resulteerende.IS ter Vergaderinge getoont ende geexhibeert een Lyste van de MagistraatsPersoonen ende Offieianten der Steede ende Lande van Willemstadt, welcke den eedt hebben gepraesteert, in gevolge haar Hoogh Mogende Resolutie van den twintighsten December seventien hondert vyftien. DD WAAE op geen resolutie is gevallen.
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [66]:
h = junksweeper.keyword_index['Holland']
h.

In [ ]:
splittekst = re.split(pattern="\s", string=unmarked_text)

In [36]:
outdir = '/Users/rikhoekstra/republic_out'

In [40]:
for yr in range(1729,1730):
    %time run_attendancelist.run(year=yr, outdir=outdir, verbose=False)

1. finding presidents
297 found
2.find provincial extraordinaris gedeputeerden
3. finding unmarked text
4. joining presidents and delegates
total 61 found
5. verifying matches
verifying spans
updating merged delegates database
6. finished to attendance lists and found database
1729 done
CPU times: user 10min 4s, sys: 4.84 s, total: 10min 9s
Wall time: 10min 36s


In [39]:
import os
os.path.join('a', 'b')

'a/b'

In [29]:
runner.moregentlemen

[16901: Bout,
 17355: de Drews,
 17821: Borre van Amerongen,
 17077: van Borssele,
 16465: van Isselmuden tot Zwollingerkamp,
 13477: van Wassenaer van Obdam,
 16377: van Rechteren tot Gramsbergen,
 12369: thoe Schwartzenberg en Hohenlansberg,
 16446: Beeldsnijder Steenbergen,
 16400: ten Brink,
 17379: Lohman,
 13278: Boner,
 17397: Quintus,
 13046: van Glinstra,
 16423: van Haersolte tot Oldenhof,
 16468: Sloet tot Lindenhorst en Slotenhagen,
 15207: van Reede tot Renswoude,
 19094: Singendonck,
 17356: Tamminga van Alberda,
 17862: van Welderen,
 17094: Velters,
 16495: Westenberg,
 12332: Vegelin van Claerbergen,
 15269: Spiering,
 17375: de Mepsche,
 12744: Banga,
 17568: van Heeckeren tot Barlham,
 15268: Brandwijk van Blokland,
 18915: van den Boetzelaar,
 17184: Musch,
 17505: van Heeckeren tot Brandsenburg,
 16374: van Broeckhuysen tot de Grote Wede,
 13998: Groeninx,
 19130: Coulman,
 19886: Cock,
 16463: van Isselmuden tot Rollecate,
 15231: van der Dussen,
 14001: de Neyn,


In [333]:
score_levenshtein_distance_ratio('Amerongen','Borre van Amerongen')

0.4736842105263158

In [34]:
twijfelresults = []
for im in runner.all_matched:
    item = runner.all_matched[im]
    if score_levenshtein_distance_ratio(item['m_kw'], item['name'])<0.6:
        twijfelresults.append(im)
        

In [36]:
len(runner.all_matched) - len(twijfelresults)

49

In [37]:
runner.all_matched[17821]

{'id': 17821,
 'm_kw': 'Amerongen',
 'score': 0.9,
 'name': 'Borre van Amerongen',
 'variants': [('Zaats van Amorongen', 'Amerongen', 0.8888888888888888),
  ('Zaats van Amerongen', 'Amerongen', 1.0),
  ('Zaats van Ameuongen', 'Amerongen', 0.8888888888888888),
  ('Jaats van Amerongen', 'Amerongen', 1.0),
  ('Amorongen', 'Amerongen', 0.8888888888888888),
  ('Amerongeu', 'Amerongen', 0.8888888888888888),
  ('Amerongen', 'Amerongen', 1.0),
  ('amerongen', 'Amerongen', 0.8888888888888888),
  ('Amervngen', 'Amerongen', 0.8888888888888888),
  ('Amexsongen', 'Amerongen', 0.6666666666666667),
  ('Amerungen', 'Amerongen', 0.8888888888888888)]}

In [38]:
def match2pattern(match:dict, bestpattern='', year=0):
    goodresults = []
    quarantine = []
    variants = match['variants']
    if bestpattern == '':
            try:
                bestpattern = max(variants, key=lambda x: x[2])[0]
            except TypeError:
                print (variant)
                raise
    if match['score']>0.8:
        results = goodresults
    else:
        results = quarantine
    for variant in variants:
        if variant[2]>0.8:
            result = {}
            result['variant_pattern'] = variant[0]
            result['variant_score'] = variant[2]
            result['variant_keyword'] = bestpattern
            result['match_keyword']=match['m_kw']
            result['match_name'] = match['name']
            result['match_score'] = match['score']
            result['match_id'] = match['id']
            results.append(result)
    return goodresults, quarantine


In [40]:
year = 1728
indexkws = []
qindexkws = []
for item in runner.all_matched:
    matches = runner.all_matched[item]
    i_kws, q_kws = match2pattern(matches, year=year)
    indexkws.extend(i_kws)
    qindexkws.extend(q_kws)

In [92]:
from collections import defaultdict
goodvariants = {}
joined_good_variants = defaultdict(list)
unique_ids = list(set([i.get('match_id') for i in indexkws]))

for k in unique_ids:
    dd = defaultdict(list)
    ikws = [d for d in indexkws if d.get('match_id') == k]
    for d in ikws: # you can list as many input dicts as you want here
        for key, value in d.items():
            if key != 'match_id':
                dd[key].append(value)
    for kie, v in dd.items():
        v = list(set(v))
        if len(v) == 1:
            v = v[0]
        dd[kie] = v
    goodvariants[k] = dd

In [93]:
goodvariants

{385: defaultdict(list,
             {'variant_pattern': ['Vis-', 'Vis'],
              'variant_score': 1.0,
              'variant_keyword': 'Vis',
              'match_keyword': 'Vis',
              'match_name': 'Visscher',
              'match_score': 0.9}),
 17927: defaultdict(list,
             {'variant_pattern': ['Poll', 'Polt', 'Poel'],
              'variant_score': 1.0,
              'variant_keyword': 'Poll',
              'match_keyword': 'Pol',
              'match_name': 'Polman',
              'match_score': 0.9}),
 16400: defaultdict(list,
             {'variant_pattern': ['Brincke',
               'Ten Brincke',
               'Brincké',
               'Brinske',
               'Jen Brincke'],
              'variant_score': 0.8333333333333334,
              'variant_keyword': 'Jen Brincke',
              'match_keyword': 'Brink',
              'match_name': 'ten Brink',
              'match_score': 1.0}),
 16403: defaultdict(list,
             {'variant_pattern': ['p

In [42]:
proposed_good_matches = []
for item in qindexkws:
    if item['variant_score'] > item['match_score']:
        item['match_keyword'] = item['variant_pattern']
        item['match_score'] = item['variant_score']
        proposed_good_matches.append(item)
        qindexkws.pop(qindexkws.index(item)) 
        

In [43]:
vrs = [i['variant_pattern'] for i in indexkws]
vids = [i['match_id'] for i in indexkws]
prids = [i['match_id'] for i in proposed_good_matches]
vids.extend(prids)

In [44]:
for item in qindexkws:
    if item['match_id'] in vids:
        print(item)

{'variant_pattern': 'Singendonek', 'variant_score': 0.9090909090909091, 'variant_keyword': 'Van Singendonck', 'match_keyword': 'Singendonck', 'match_name': 'Singendonck', 'match_score': 0.0, 'match_id': 19094}
{'variant_pattern': 'Singendonck', 'variant_score': 1.0, 'variant_keyword': 'Van Singendonck', 'match_keyword': 'Singendonck', 'match_name': 'Singendonck', 'match_score': 0.0, 'match_id': 19094}
{'variant_pattern': 'Singeudonck', 'variant_score': 0.9090909090909091, 'variant_keyword': 'Van Singendonck', 'match_keyword': 'Singendonck', 'match_name': 'Singendonck', 'match_score': 0.0, 'match_id': 19094}
{'variant_pattern': 'Velters', 'variant_score': 1.0, 'variant_keyword': 'Velters', 'match_keyword': 'Velters', 'match_name': 'Velters', 'match_score': 0.0, 'match_id': 17094}
{'variant_pattern': 'zelaar', 'variant_score': 0.8333333333333334, 'variant_keyword': 'Velters', 'match_keyword': 'Velters', 'match_name': 'Velters', 'match_score': 0.0, 'match_id': 17094}
{'variant_pattern': '

In [45]:
len(qindexkws)

26

In [46]:
len(proposed_good_matches)

27

In [48]:
from statistics import mean
mean([i.get('match_score') for i in indexkws])

0.9717391304347827

In [49]:
min([i['match_score'] for i in indexkws])

0.9

In [52]:
from republic.republic_keyword_matcher.finders.identify import iterative_search
reslt = {}
for i in twijfelresults:
    item = runner.all_matched[i]
    l = 0
    v = ''
    v = max(item['variants'], key=lambda z: len(z[0]))[0]
    reslt[v] = iterative_search(name=v, year=1728, debug=False, df=runner.abbreviated_delegates)

In [55]:
from republic.republic_keyword_matcher.finders.identify import identify

In [56]:
iterative_search(name='Taats', year=1728, df=runner.abbreviated_delegates)

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
127,466,Taay,1753-01-01,1799-12-31,NaN,Staten-Generaal der Verenigde Nederlandse Prov...,"gedeputeerde, lid","PeriodIndex(['1795-02-04', '1795-02-05', '1795...",True,True,"[1795, 1796]","[1753, 1799]",0.7
2629,14074,Cats,1577-01-01,1660-12-31,NaN,Gecommitteerde Raden van Holland in het Zuider...,"raadpensionaris, ordinaris gedeputeerde","PeriodIndex(['1629-09-19', '1629-09-20', '1629...",True,True,"[1629, 1651]","[1577, 1660]",0.7


In [57]:
from IPython.core.display import display, HTML
for i in reslt:
    display(i, reslt[i])

'Zaats van Amorongen'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
144,483,Staal van Ingen,1755-01-01,1804-12-31,NaN,Staten-Generaal der Verenigde Nederlandse Prov...,gedeputeerde,"PeriodIndex(['1796-02-04', '1796-02-05', '1796...",True,True,"[1796, 1796]","[1755, 1804]",0.7
6161,17821,Borre van Amerongen,None,1640-01-21,NaN,"Ridderschap van Utrecht, Staten-Generaal der V...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1606-01-01', '1606-01-02', '1606...",True,True,"[1606, 1640]","[1562, 1640]",0.7


'Van lsselmuden'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
7281,19091,van Essen Essen,None,None,NaN,Staten-Generaal der Verenigde Nederlanden (158...,ordinaris gedeputeerde,"PeriodIndex(['1713-05-01', '1713-05-02', '1713...",True,True,"[1713, 1716]","[1669, 1738]",0.7


'Jan MWiassenaer'

...

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
1260,12531,Velsius,None,None,NaN,"Raad van Leeuwarden, Gedeputeerde Staten van F...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1610-01-01', '1610-01-02', '1610...",True,True,"[1610, 1615]","[1566, 1637]",0.7
5793,17347,Pels,None,1705-06-22,NaN,Staten-Generaal der Verenigde Nederlanden (158...,"ordinaris gedeputeerde, extraordinaris gecommi...","PeriodIndex(['1680-02-24', '1680-02-25', '1680...",True,True,"[1680, 1705]","[1636, 1705]",0.7
7267,19074,Pels,None,None,NaN,Staten-Generaal der Verenigde Nederlanden (158...,ordinaris gedeputeerde,"PeriodIndex(['1711-05-01', '1711-05-02', '1711...",True,True,"[1711, 1715]","[1667, 1737]",0.7


'uytstel'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
6480,18200,Buttel,None,None,NaN,"Gedeputeerde Staten van Groningen (1595-1795),...","lid, gecommitteerde","PeriodIndex(['1598-03-02', '1598-03-03', '1598...",False,False,"[1598, 1602]","[1564, 1624]",0.3


'verhaalen'

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,score
2600,14044,Vermeulen,1732-01-01,1810-06-12,NaN,"Vroedschap van Haarlem, Staten-Generaal der Ve...","lid, ordinaris gedeputeerde, gecommitteerde","PeriodIndex(['1761-01-01', '1761-01-02', '1761...",True,True,"[1761, 1795]","[1732, 1810]",0.7


G_heren = nx.Graph()
d_nodes = sorted(cl_heren)
for node in d_nodes:
    attached_nodes = cl_heren[node]
    G_heren.add_node(node)
    for nod in attached_nodes:
        G_heren.add_edge(node, nod)
pm_heren = list(nx.connected_components(G_heren))
pm_heren

In [58]:
item

{'id': 15267,
 'm_kw': 'Overhagen',
 'score': 0.8,
 'name': 'Schagen',
 'variants': [('verhaalen', 'Overhagen', 0.6666666666666667),
  ('verRaaden', '', 0)]}

In [172]:
from collections import Counter
lastfragments = []
lastpatterns = []
for ob in runner.searchobs:
    lastknownfragment = ''
    lastpattern = ''
    fragments = runner.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        f = item.get('delegate_name') or ''
        p = item.get('pattern') or ''
        if  f != '':
            lastknownfragment = item['delegate_name']
            lastfragments.append(lastknownfragment)
            lastpattern = p
            lastpatterns.append(lastpattern)
            break
Counter(lastfragments).most_common(10)

[('thoe Schwartzenberg en Hohenlansberg', 67),
 ('ten Brink', 49),
 ('van Isselmuden tot Zwollingerkamp', 27),
 ('Tamminga van Alberda', 17),
 ('Lohman', 15),
 ('van Haersolte tot Oldenhof', 14),
 ('van Rechteren tot Gramsbergen', 14),
 ('Scriverius', 12),
 ('Banga', 7),
 ('Borre van Amerongen', 6)]

In [118]:
for pers in Counter(lastfragments).most_common(10):
    print(pers, [goodvariants[d] for d in goodvariants if goodvariants[d]['match_name'].lower() == pers[0].lower()])
    #print(pers, [runner.all_matched[d] for d in runner.all_matched if runner.all_matched[d]['name'].lower() == pers[0].lower()])

('thoe Schwartzenberg en Hohenlansberg', 57) [defaultdict(<class 'list'>, {'variant_pattern': ['Van Schwartzenbergh', 'Schawartzenbergb', 'Schwartzxenbergb', 'Schwarizenbergh', 'Sehwartzenbergh', 'Schwartzenbergbh', 'Van Schwartzenhergh', 'Van Schwartzenbergb', 'Sehwartzenbergbh', 'Jan Schwart zenbergt', 'Van Sehwartzenbergh', 'Schwartzenbergir', 'Schwarizenbergb', 'Van Schwart zenbergh', 'Schwartzenberg', 'Schavartzenbergh', 'an Schwartzenbergh', 'Schwartzenbergb', 'Schwartzenbergh', 'Schwartzenberghb'], 'variant_score': [0.9333333333333333, 1.0, 0.9285714285714286, 0.8666666666666667], 'variant_keyword': 'Van Schwartzenbergh', 'match_keyword': 'Schwartzenberg', 'match_name': 'thoe Schwartzenberg en Hohenlansberg', 'match_score': 1.0, 'name': []})]
('ten Brink', 49) [defaultdict(<class 'list'>, {'variant_pattern': ['Brincke', 'Ten Brincke', 'Brincké', 'Brinske', 'Jen Brincke'], 'variant_score': 0.8333333333333334, 'variant_keyword': 'Jen Brincke', 'match_keyword': 'Brink', 'match_name

In [173]:
common_last_delegates = [pers[0] for pers in Counter(lastfragments).most_common(10)]

In [174]:
from republic.fuzzy.fuzzy_keyword_searcher import FuzzyKeywordSearcher
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}
fks = FuzzyKeywordSearcher(config=fuzzysearch_config)
fks.use_word_boundaries = False
kws = [goodvariants[p]['match_keyword'] for p in goodvariants]
false_kws = ['PRAESIDE','PRAESENTIBUS', 'Nihil Actum']
kws.extend(false_kws)
fks.index_keywords(kws)
for kw in kws:
    if kw not in false_kws:
        z = [goodvariants[d] for d in goodvariants if goodvariants[d]['match_keyword'] == kw][0]
        vrs = z["variant_pattern"]
        for v in vrs:
            fks.index_spelling_variant(kw, v)

In [181]:
suspect_obs = defaultdict(list)
for ob in runner.searchobs:
    fragments = runner.searchobs[ob].get_fragments()
    fragments.reverse()
    for item in fragments:
        lastdelegate = ''
        ld = item.get('delegate_name')
        if ld in common_last_delegates:
            break
        elif ld == '': 
            p = item.get('pattern')
            r = fks.find_candidates(text=p)
            if len(r)>0:
                r = max(r, key=lambda x: x['levenshtein_distance'])
                if r["match_keyword"] in false_kws:
                    suspect_obs[r["match_keyword"]].append(ob)
                elif r["match_keyword"] in commonlastgoodvariantsvs:
                    begin = item['begin']
                    changeob = runner.searchobs[ob]
                    begin = res[-2]
                    for span in changeob.matched_text.spans:
                        if span.begin>=begin:
                            changeob.matched_text.spans.remove(span)

In [182]:
suspect_obs

defaultdict(list,
            {'PRAESENTIBUS': ['session-1728-01-26-num-1',
              'session-1728-01-27-num-1',
              'session-1728-01-31-num-1',
              'session-1728-02-02-num-1',
              'session-1728-02-05-num-1',
              'session-1728-02-17-num-1',
              'session-1728-03-02-num-1',
              'session-1728-03-10-num-1',
              'session-1728-03-13-num-1',
              'session-1728-05-15-num-1',
              'session-1728-06-30-num-1',
              'session-1728-08-16-num-1',
              'session-1728-08-30-num-1',
              'session-1728-12-27-num-1'],
             'PRAESIDE': ['session-1728-02-17-num-1',
              'session-1728-08-30-num-1'],
             'Nihil Actum': ['session-1728-08-30-num-1']})

In [179]:
for i in suspect_obs:
    print(runner.searchobs[i].get_fragments())

[{'type': 'unmarked', 'pattern': '1728. ', 'end': 0, 'begin': 5}, {'type': 'delegate', 'idnr': 0, 'pattern': 'kan', 'begin': 6, 'end': 9, 'delegate_id': 13287, 'delegate_name': 'van Canter', 'delegate_gewest': '', 'delegate_score': 0.0}, {'type': 'unmarked', 'pattern': ' werden getreden. ', 'end': 9, 'begin': 26}, {'type': 'pre', 'idnr': 0, 'pattern': 'PRAESIDE, Den Heere', 'begin': 27, 'end': 46, 'delegate_id': 0, 'delegate_name': '', 'delegate_gewest': 0, 'delegate_score': 0}, {'type': 'president', 'idnr': 0, 'pattern': 'Van Wassenaer', 'begin': 47, 'end': 60, 'delegate_id': 13477, 'delegate_name': 'van Wassenaer van Obdam', 'delegate_gewest': '', 'delegate_score': 0.0}, {'type': 'unmarked', 'pattern': '. ', 'end': 60, 'begin': 61}, {'type': 'presentibus', 'idnr': 0, 'pattern': 'PRAESENTIBUS', 'begin': 62, 'end': 74, 'delegate_id': 0, 'delegate_name': '', 'delegate_gewest': 0, 'delegate_score': 0}, {'type': 'unmarked', 'pattern': ', De Heeren Van Lynden, van ', 'end': 74, 'begin': 10

In [169]:
begin = res[-2]
for span in changeob.matched_text.spans:
    if span.begin>=begin:
        changeob.matched_text.spans.remove(span)
changeob.matched_text.spans

[fragment type preamble: 0-31,
 fragment type pre: 32-51,
 fragment type president: 52-71,
 fragment type presentibus: 73-85,
 fragment type delegate: 113-122,
 fragment type delegate: 131-146,
 fragment type delegate: 163-173,
 fragment type delegate: 178-188,
 fragment type delegate: 198-205,
 fragment type delegate: 224-233,
 fragment type delegate: 255-267,
 fragment type delegate: 272-282,
 fragment type delegate: 283-288,
 fragment type delegate: 289-294,
 fragment type delegate: 316-326,
 fragment type delegate: 331-341,
 fragment type delegate: 346-355,
 fragment type delegate: 360-371,
 fragment type delegate: 376-384]

In [152]:
res[2]

{'match_keyword': 'Tamminga',
 'match_term': 'Tamminga',
 'match_string': 'Tamminga',
 'match_offset': 0,
 'char_match': 1.0,
 'ngram_match': 1.0,
 'levenshtein_distance': 1.0}

In [141]:
commonlastgoodvariants = {v:goodvariants[v] for v in goodvariants if goodvariants[v]['match_name'] in common_last_delegates}

In [147]:
commonlastgoodvariantsvs = []
for v in goodvariants:
    if goodvariants[v]['match_name'] in common_last_delegates:
        commonlastgoodvariantsvs.extend(goodvariants[v]['variant_pattern'])
commonlastgoodvariantsvs

['Brincke',
 'Ten Brincke',
 'Brincké',
 'Brinske',
 'Jen Brincke',
 'Haarsolte',
 'Van Haarsolte',
 'De Drews',
 'Drews',
 'Zamminga',
 'JTamminga',
 'Van Zamminga',
 'Tamminga',
 'Tammiaga',
 'Van Tamminga',
 'Ilsselmuden',
 'Van lsselmuden',
 'lsselmuden',
 'Vun lsselmuden',
 'ssselmuden',
 'Jsselmuden',
 'Isselmuden',
 'Jan Jsselmuden',
 'Van Schwartzenbergh',
 'Schawartzenbergb',
 'Schwartzxenbergb',
 'Schwarizenbergh',
 'Sehwartzenbergh',
 'Schwartzenbergbh',
 'Van Schwartzenhergh',
 'Van Schwartzenbergb',
 'Sehwartzenbergbh',
 'Jan Schwart zenbergt',
 'Van Sehwartzenbergh',
 'Schwartzenbergir',
 'Schwarizenbergb',
 'Van Schwart zenbergh',
 'Schwartzenberg',
 'Schavartzenbergh',
 'an Schwartzenbergh',
 'Schwartzenbergb',
 'Schwartzenbergh',
 'Schwartzenberghb',
 'Van Rechtereu',
 'Van Rechteren',
 'Jan Rechteren',
 'Rechteren',
 'Rechtereu',
 'Rechieren']

In [457]:
lastspans = [f[-1].to_json() for f in out]

KeyError: -1

In [465]:
lastfragments[-100]a

{'type': 'unmarked', 'pattern': '', 'begin': 240, 'end': 269}

In [23]:
spans = []
for T in ra.searchobs:
    ob = ra.searchobs[T]
    spans.append(ob.get_spans())

In [40]:
noprez = {}
for T in ra.searchobs:
    ob = ra.searchobs[T]
    p = [s for s in ob.get_spans() if s.type=='president']
    if p:
        noprez[T] = p
    else:
        noprez[T] = None


In [54]:
upr = list(set(ra.presidents))

['Velters',
 'Jan MWiassenaer',
 'Ten Brincke',
 'Mey',
 'Quint',
 'Van Renswoude',
 'Zaats van Amerongen',
 'Van Borselen',
 'Van Renswonde',
 'Jan Wassenaer',
 'Jan Schwart zenbergt',
 'Van Tamminga',
 'an Glinstra',
 'De Drews',
 'Van Sehwartzenbergh',
 'Jan Rechteren',
 'Lohman',
 'De Dreus',
 'Van lielmuden',
 'De Dreuws',
 'Boat',
 'an Wassenaer',
 'Van Wassenaer',
 'Vun lsselmuden',
 'Jan Borsselen',
 'Lobman',
 'Zaats van Ameuongen',
 'Van Schwart zenbergh',
 'Van Schwartzenbergh',
 'Zaats van Amorongen',
 'an Schwartzenbergh',
 'Jan Lynden',
 'Van Haarsolte',
 'Jen Brincke',
 'Bout',
 'Jaats van Amerongen',
 'Van Singendonck',
 'Van lselmuden',
 'Van Zamminga',
 'Van Glinstra',
 'De Drewt',
 'Van Lynden',
 'Van Schwartzenbergb',
 'Van Steenbergen',
 'Woey',
 'Van Borsselen',
 'Noey',
 'Van Yassenaer',
 'Van Schwartzenhergh',
 'Jan Jsselmuden',
 'Van lsselmuden',
 'Van Rechteren',
 'Van Hassenaer',
 'Van Haarsoste',
 'Van Welderen',
 'Vau Wassenaer']

In [51]:
ks = list(noprez.keys())
ks.sort()


In [65]:
noprez[t][0]

fragment type president: 51-71

In [106]:
fks.keyword_index.keys()

dict_keys(['Bout', 'de Drews', 'Borre van Amerongen', 'van Borssele', 'van Isselmuden tot Zwollingerkamp', 'van Lynden', 'van Wassenaer van Obdam', 'van Rechteren tot Gramsbergen', 'thoe Schwartzenberg en Hohenlansberg', 'ten Brink', 'Lohman', 'Nobel', 'Quintus', 'van Glinstra', 'van Haersolte tot Oldenhof', 'van Reede tot Renswoude', 'Singendonck', 'Tamminga van Alberda', 'van Welderen', 'Velters', 'van Wijnbergen tot de Glinthorst', 'Westenberg', 'Vegelin van Claerbergen', 'Weveringh', 'de Mepsche', 'Banga', 'van Heeckeren tot Barlham', 'van Deurn', 'Brandwijk van Blokland', 'van den Boetzelaar', 'Musch', 'van Heeckeren tot Brandsenburg', 'Fontein', 'Groeninx', 'Cock', 'Coulman', 'van der Dussen', 'Eelbo', 'van Maneel', 'van den Eede', 'Hagen', 'Verkerk', 'van Goslinga', 'Greven', 'Cramer', 'de Groot', 'van Heeckeren', 'Hendrickx', 'van Hoeckelom', 'Hoogerbeets', 'van Borssele van der Hooghe', 'van Huss tot Rees', 'Sloet tot Lindenhorst en Slotenhagen', 'van der Jagt', 'Torck', 'van 

In [33]:
# this does not generalize much, but we'll leave that for later
from collections import defaultdict
from republic.republic_keyword_matcher.finders.pattern_finders import make_herensearcher
from republic.fuzzy.fuzzy_keyword_searcher import score_levenshtein_distance_ratio


class DelegateChecker(object):
    def __init__(self, keywords, searchobs):
        self.searcher = make_herensearcher(keywords=keywords)
        self.context_searcher = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
        self.searchobs = searchobs
        self.likely_prez = defaultdict(list)
        self.checkdeps = self.make_noprez()
        self.ks = self.make_keys()
        
    def make_keys(self):
        ks = list(self.checkdeps.keys())
        ks.sort() # we depend on the key order
        return ks
        
    def get_vicinity_presidents(self,t):
        mn = max(self.ks.index(t)-7, 0) 
        mx = min(self.ks.index(t)+7, len(ks)-1)
        likely_prez = defaultdict(list)
        for pt in ks[mn:mx]:
            if self.checkdeps[pt]:
                president = self.checkdeps[pt][0]
                name = president.delegate_name
                pattern = president.pattern
                likely_prez[name].append(pattern)
        return likely_prez
        # for p in likely_prez:
        #     ra.all_matched
#         xsisting_kws = list(self.searcher.keyword_index.keys())
#         kws = [k for k in list(likely_prez.keys()) if k not in xsisting_kws]            
#         self.searcher.index_keywords(kws)
#         for k in kws:
#             for variant in likely_prez[k]:
#                 self.searcher.index_spelling_variant(k, variant=variant)
    
    def check(self):
        results = []
        for t in self.ks:
            if not self.checkdeps[t]:
                txt = self.searchobs[t].text
                if len(txt) > 10:
                    res = s.find_candidates(text=txt)
                    if len(res) == 0:
                        break
                    hres = [r for r in res if r['match_keyword']=='Den Heere']
                    sres = min(hres, key=lambda x: x['match_offset'])
                    if len(sres) > 0:
                        start = sres['match_offset']  + len(sres['match_string'])
                    else:
                        start = 0
                    pres = [r for r in res if r['match_keyword']=='PRAESENTIBUS']
                    if len(pres) > 0:
                        eres = min(pres, key=lambda x: x['match_offset'])
                        end = eres['match_offset']
                    else:
                        end = start + 20
                    stxt = txt[start:end]
                    stxt = stxt.strip()
                likely_prez = self.get_vicinity_presidents(t)
                res = {}
                for cand in likely_prez:
                    candidate = likely_prez[cand]
                    for i in candidate:
                        res[i] = score_levenshtein_distance_ratio(i, stxt)
                chosen = max(res, key=lambda x: res[x])
                score = res[chosen]
                if score > 0.6:
#                     for cand in likely_prez:
#                         candidate = max([(i, score_levenshtein_distance_ratio(stxt, i)) for i in likely_prez[cand]])
                    candidate = chosen
                else:
                    prop = self.searcher.find_candidates(stxt)
                    if len(prop)>0:
                        candidate = max(prop, key=lambda x: x['levenshtein_distance'])
                    else:
                        candidate = None
                results.append({'txt': stxt,
#                                'lp':likely_prez,
                                'search result': candidate}) 
        return results

    
    def make_noprez(self):
        noprez = {}
        for T in self.searchobs:
            ob = self.searchobs[T]
            p = [s for s in ob.get_spans() if s.type=='president']
            if p:
                noprez[T] = p
            else:
                noprez[T] = None
        return noprez

In [244]:
s = make_herensearcher(keywords={'PRAESENTIBUS':[], 'Den Heere':[]})
res = [r for r in s.find_candidates(keyword="Den Heere", text="Den Heere Jan Steenbergen. PRAZSENTIBUS, De Heeren Van Lynden, van Singeudonck, van Heeckeren tot de") if r['match_keyword']=='Den Heere']
sres = min(res, key=lambda x: x['match_offset'])
sres['match_offset']  + len(sres['match_string'])

9

In [245]:
keywords = {i.name:[v.form for v in i.variants['general']] for i in ra.moregentlemen}


In [246]:
keywrds = {}
for e in ra.all_matched:
    i = ra.all_matched[e]
    nm = i["name"]
    f = [i["m_kw"]]
    f.extend([v[1] for v in i["variants"]])
    keywrds[nm]=list(set(f))
    

In [247]:
keywrds['Tamminga van Alberda']

['Tamminga', 'Vlaming', 'Tamming']

In [322]:
dlgchk = DelegateChecker(keywords=keywrds, searchobs=ra.searchobs)

In [249]:
dlgchk.searcher.find_candidates("Van Schwartzenbergh. ")

[]

In [251]:
d = {'Velters': ['Velters', 'Velters', 'Velters', 'Velters'], 'van Wassenaer van Obdam': ['Van Wassenaer', 'Van Wassenaer'], 'van Reede tot Renswoude': ['Van Renswoude', 'Van Renswoude', 'Van Renswonde']}

In [256]:
txt = ' Van Renswoude.'
res = {}
for cand in d:
    candidate = d[cand]
    for i in candidate:
        res[i] = score_levenshtein_distance_ratio(i, txt)

In [259]:
max(res, key=lambda x: res[x])

'Van Renswoude'

In [323]:
dlgchk.check()

[{'txt': 'Vesters.', 'search result': 'Velters'},
 {'txt': 'Velters.', 'search result': 'Velters'},
 {'txt': 'Van Renswoude.', 'search result': 'Van Renswoude'},
 {'txt': 'Van Schwartzenbergh.', 'search result': 'Van Schwart zenbergh'},
 {'txt': 'Van Tamminga.', 'search result': 'Van Tamminga'},
 {'txt': 'Van Lynden.', 'search result': 'Van Lynden'},
 {'txt': 'Zaats van Amerongen.', 'search result': 'Zaats van Amerongen'},
 {'txt': 'Zaats van Amerongen.noch voor tien eerstkomende jaaren van de',
  'search result': {'match_keyword': 'Borre van Amerongen',
   'match_term': 'Borre van Amerongen',
   'match_string': 'Zaats van Amerongen',
   'match_offset': 0,
   'char_match': 0.7368421052631579,
   'ngram_match': 0.7,
   'levenshtein_distance': 0.736842105263158}},
 {'txt': 'Van Schwartzenbergh.', 'search result': 'Van Schwartzenbergh'},
 {'txt': 'Van Schwart zenbergh.', 'search result': 'Van Schwartzenbergh'},
 {'txt': 'Van Schwartzenbergb.', 'search result': 'Van Schwartzenbergh'},
 {'t

In [317]:
len("'Van Lynden, van Singendonck, IS ter Vergaderinge gelesen de Requeste van Johan Coenraadts Hendricks")

100

In [276]:
score_levenshtein_distance_ratio('Van Schwartzenbergh', 'Van Schwart zenbergh')

0.95

In [104]:

fks = make_herensearcher(keywords={i.name:[v.form for v in i.variants['general']] for i in ra.moregentlemen})

In [ ]:
make_herensearcher(keywords={})

In [83]:
for t in ks:
    if not noprez[t]:
        likely_prez = get_vicinity_presidents(noprez, t, )
        txt = ra.searchobs[t].text
        if len(txt) > 10:
            e = txt.find('PRAESENTIBUS')
            if e > -1:
                stxt = txt[:e]
            else: 
                stxt = txt[:100]
            print(likely_prez.find_candidates(stxt), stxt)
                
         

[{'match_keyword': 'Velters', 'match_term': 'Velters', 'match_string': 'Vesters', 'match_offset': 10, 'char_match': 0.8571428571428571, 'ngram_match': 0.75, 'levenshtein_distance': 0.8571428571428572}] Den Heere Vesters. 
[{'match_keyword': 'Velters', 'match_term': 'Velters', 'match_string': 'Velters', 'match_offset': 10, 'char_match': 1.0, 'ngram_match': 1.0, 'levenshtein_distance': 1.0}] Den Heere Velters. 
[{'match_keyword': 'van Reede tot Renswoude', 'match_term': 'van Reede tot Renswoude', 'match_string': 'Den Heere Van Renswoude', 'match_offset': 0, 'char_match': 0.7391304347826086, 'ngram_match': 0.5833333333333334, 'levenshtein_distance': 0.6956521739130435}] Den Heere Van Renswoude. 
[] Den Heere Van Schwartzenbergh. 
[] Den Heere Van Tamminga. 
[{'match_keyword': 'van Lynden', 'match_term': 'van Lynden', 'match_string': 'Van Lynden', 'match_offset': 10, 'char_match': 0.9, 'ngram_match': 0.8181818181818182, 'levenshtein_distance': 0.9}] Den Heere Van Lynden. 
[] Den Heere Zaat

In [81]:
likely_prez

In [ ]:
for k in ra.searchobs:
    so = ra.searchobs[k]
    sspans = so.get_spans()
    for s in sspans:
        if s.pattern in s.delegate_name:
            if s.delegate_name > s.pattern
            so.get_txt

In [102]:
ra.all_matched

{16901: {'id': 16901,
  'm_kw': 'Bout',
  'score': 0.0,
  'name': 'Bout',
  'variants': [('Bout', 'Bout', 1.0), ('Boat', 'Bodt', 0.75)]},
 17355: {'id': 17355,
  'm_kw': 'Drews',
  'score': 1.0,
  'name': 'de Drews',
  'variants': [('De Drews', 'Drews', 1.0),
   ('De Dreus', 'Drews', 0.8),
   ('De Dreuws', 'Drews', 0.6),
   ('De Drewt', 'Drews', 0.8),
   ('Drewt', 'Drews', 0.8),
   ('Drews', 'Drews', 1.0)]},
 17821: {'id': 17821,
  'm_kw': 'Amerongen',
  'score': 0.9,
  'name': 'Borre van Amerongen',
  'variants': [('Zaats van Amorongen', 'Amerongen', 0.8888888888888888),
   ('Zaats van Amerongen', 'Amerongen', 1.0),
   ('Jaats van Amerongen', 'Amerongen', 1.0),
   ('Zaats van Ameuongen', 'Amerongen', 0.8888888888888888),
   ('Ordre', 'Orden', 0.6),
   ('ordre', 'Borre', 0.6),
   ('ordres', 'Borre', 0.6)]},
 17077: {'id': 17077,
  'm_kw': 'Borssele',
  'score': 1.0,
  'name': 'van Borssele',
  'variants': [('Van Borselen', 'Borselen', 1.0),
   ('Jan Borsselen', 'Borssele', 1.0),
   ('V

vars2graph(cl_heren) 

existing_herensearcher = FuzzyKeywordSearcher(config=fuzzysearch_config)
existing_herensearcher.index_keywords(pm_heren)

In [4]:
fuzzysearch_config = {
    "char_match_threshold": 0.8,
    "ngram_threshold": 0.6,
    "levenshtein_threshold": 0.5,
    "ignorecase": False,
    "ngram_size": 2,
    "skip_size": 2,
}

## retrieve presentielijsten and convert to objects

In [6]:
searchobs = make_presentielijsten(year='1728')

In [7]:
searchobs['session-1728-01-01-num-1']

Dominica den 4. Januarii 1728. Nihil actum est. Luna den 5. Januari 1728. PRAESIDE, Den Heere Velters. PRAESENTIBUS, De Heeren Van Singendonck, van Heeckeren tot Barlbam, van Heeckeren tot den Brandt zenborgh, van U'wbergen, Torck, Umbgroeven, met twee extraordinaris Gedeputeerden uyt de Provincie van Gelderlandt. Van Wassenaer, vander Dussen, Bors van Waveren, vau Marseveen, de Raadt. Noey, van Hoorn, met een extraordinaris Gedeputeerde uyt de Provincie van Zeelandt. Van Renswoude. Van Schwartzenberg, Vegilin. Van Haarsolte, van lsselmuden. Van Tamminga.DE Resolutien eergisteren genomen , zyn gelesen ende geresumeert, gelyck oock geresumeert ende gearresteert zyn de Depesches daar uytresulterende.

### 1. Find presidents

In [8]:
presidents = president_searcher(presentielijsten=searchobs)

In [9]:
heren = presidents


In [10]:
heren = [h.strip() for h in heren]

### 2.find provincial extraordinaris gedeputeerden

In [11]:
ps = province_searcher(presentielijsten=searchobs)

### 3. make a list of remaining words from the attendents lists

This collects all non-'identified words' from the searchobjects

In [12]:
unmarked = make_groslijst(presentielijsten=searchobs)

In [13]:
# filter out words that occur less than 5 times 
from collections import Counter
c = Counter(unmarked)
len([k for k in c if c[k]>3])

513

In [14]:
filtered_text = " ".join(list(c.keys()))

In [15]:
fks = FuzzyKeywordSearcher(fuzzysearch_config)
tussenkeys = fks.find_close_distance_keywords(list(c.keys()))


In [16]:
dralist = vars2graph(tussenkeys)


In [17]:
with open('republic_junk.json','r') as rj:
    ekwz = json.load(fp=rj)
provincies=['Holland','Zeeland','West-Vriesland','Gelderland','Overijssel', 'Utrecht','Friesland']
from republic.model.republic_phrase_model import week_day_names, month_names_early, month_names_late, spelling_variants
months = month_names_early + month_names_late 
junksweeper = FuzzyKeywordSearcher(config=fuzzysearch_config)
junksweeper.index_keywords(list(ekwz.keys()))
for key in ekwz.keys():
    for variant in ekwz[key]:
        junksweeper.index_spelling_variant(keyword=key, variant=variant)
junksweeper.index_keywords(months)
junksweeper.index_keywords(provincies)

In [18]:
def get_lscore(r):
    return r['levenshtein_distance']

rawres = []
for t in dralist:
#    t = rawlist[i]
    if len(t)>1:
        rawtext = ' '.join(t)
    else: 
        rawtext = t[0]
    r = junksweeper.find_candidates(rawtext)
    try:
        nr = max(r, key=get_lscore)
        if nr['levenshtein_distance'] < 0.5:
            rawres.append(t)
    except ValueError:
        rawres.append(t)

In [19]:
#deputies = fks.find_close_distance_keywords(rawlist)
deputies = rawres

We now have:

1. __heren__: a list of presidents
1. __ezkw__: a rough list of words that have been identified as junk before
1. __deputies__: a rough list of words that are presumably delegates

### 5. cluster them into connected variants. Ideally these would contain the different variants of deputies

In [20]:
cl_heren = fks.find_close_distance_keywords(heren)
G_heren = nx.Graph()
d_nodes = sorted(cl_heren)
for node in d_nodes:
    attached_nodes = cl_heren[node]
    G_heren.add_node(node)
    for nod in attached_nodes:
        G_heren.add_edge(node, nod)
#list(nx.connected_components(G_heren))

#which clusters them nicely

### 6. Identify the attendance lists with this overview of deputies 

In [21]:
abbreviated_delegates = pd.read_pickle('sheets/abbreviated_delegates.pickle')

In [22]:
#previously_matched = pd.read_excel('sheets/xml_herkend.xlsx')
previously_matched = pd.read_pickle('sheets/found_deputies.pickle')
previously_matched['id'] = previously_matched['ref_id']
previously_matched.columns

pm_heren = list(previously_matched['name'].unique())

In [23]:
TUSSENVOEGSELS = [
    'à',
    '\xe0',
    r'd\'',
    r'de',
    r'den',
    r'der',
    r'des',
    r'di',
    r'en', 
    r'het',
    r"in 't",
    r'la',
    r'la', 
    r'le',
    r'of',
    r'van',
    r'ten', 
    r'thoe', 
    r'tot', 
    r"'t",
]


In [24]:
stopwords = TUSSENVOEGSELS
def nm_to_delen(naam, stopwords=stopwords):
    nms = [n for n in naam.split(' ') if n not in stopwords]
    #nms.append(naam)
    return nms
           
keywords = list(abbreviated_delegates.name)
kwrds = {key:nm_to_delen(key) for key in keywords}


In [25]:
nwkw = {d:k for k in list(set(keywords)) for d in k.split(' ') if d not in stopwords}

In [26]:
# note that this config is more lenient than the default values 
fuzzysearch_config = {'char_match_threshold': 0.7,
 'ngram_threshold': 0.5,
 'levenshtein_threshold': 0.5,
 'ignorecase': False,
 'ngram_size': 2,
 'skip_size': 2}

In [27]:
herensearcher = FuzzyKeywordSearcher(config=fuzzysearch_config)
exclude = ['Heeren', 'van', 'met', 'Holland']
filtered_kws = [kw for kw in nwkw.keys() if kw not in exclude]
herensearcher.index_keywords(keywords=filtered_kws)
for k in filtered_kws:
    for variant in nwkw[k]:
        herensearcher.index_spelling_variant(k,variant=variant)

In [28]:
from collections import defaultdict
transposed_graph = defaultdict(list)
for node, neighbours in kwrds.items():
    for neighbour in neighbours:
        transposed_graph[neighbour].append(node)

In [29]:
day = 1728
year = day

In [30]:

dayinterval = pd.Interval(day, day, closed="both")
register = {}
pats = []

In [31]:
# setup presidents for matching
existing_herensearcher = FuzzyKeywordSearcher(config=fuzzysearch_config)
existing_herensearcher.index_keywords(pm_heren)
connected_presidents = list(nx.connected_components(G_heren))
matchfnd = FndMatch(year=year,
                 #patterns=pats,
                 register=register,
                 rev_graph=transposed_graph,
                 searcher=herensearcher,
                 junksearcher=junksweeper,
                 df=abbreviated_delegates)

In [32]:
found_presidents = find_delegates(input=connected_presidents,
                                  matchfnd=matchfnd,
                                  df=abbreviated_delegates,
                                  previously_matched=previously_matched,
                                  year=1728)

### Cluster deputies

We now do the same trick with the delegates. 

In [33]:
found_delegates = find_delegates(input=deputies,
                                 matchfnd=matchfnd,
                                 df=abbreviated_delegates,
                                 previously_matched=previously_matched,
                                 year=1728 )

### join presidents and deputies


In [34]:
all_matched = {}
for d in [found_presidents['matched'], found_delegates['matched']]:
    for key in d:
        if key not in all_matched.keys():
            all_matched[key]= d[key]
        else:
            all_matched[key]['variants'].extend(d[key]['variants'])

In [35]:
moregentlemen = [MatchHeer(all_matched[d]) for d in all_matched.keys() if type(d)==int] # strange keys sneak in

In [36]:
mh_df = pd.DataFrame(all_matched).transpose().reset_index()

In [37]:
matchsearch = FuzzyKeywordSearcher(config=fuzzysearch_config)
kws = defaultdict(list)
matcher = {}
for key in all_matched:
    variants = all_matched[key].get('variants')
    keyword = all_matched[key].get('m_kw')
    idnr = all_matched[key].get('id')
    name = all_matched[key].get('name')
    for variant in variants:
        kws[keyword].append(variant[0])
        matcher[variant[0]] = {'kw':keyword, 'id':idnr, 'name':name}
matchsearch.index_keywords(list(kws.keys()))
for k in kws:
    for v in kws[k]:
        matchsearch.index_spelling_variant(keyword=k,variant=v)

In [38]:
deputyregister = defaultdict(list)
search_results = {}
# fm = FndMatch(year=1728,
#                  #patterns=pats,
#                  register=register,
#                  rev_graph=transposed_graph,
#                  searcher=herensearcher,
#                  junksearcher=junksweeper,
#                  df=abbreviated_delegates)
for T in searchobs:
    ob = searchobs[T].matched_text
    mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched, match_search=matchsearch)
#     for span in ob.spans:
#         if span.type in ['president', 'delegate']:
#             if span.pattern == '':
#                 span.set_pattern(pattern=ob.item[span.begin:span.end])
#             r = fm.match_candidates(heer=span.pattern)
#             span.set_delegate(delegate_id=r.id, 
#                               delegate_name=r.proposed_delegate, 
#                               delegate_score=r.score)

### 8. Mark delegates 

In [39]:
delresults = Counter()
for T in searchobs.keys():
    searchob = searchobs[T]
    result = get_delegates_from_spans(searchob.matched_text)
    try:
        id  = result['delegate']['id']
        if id:
            delresults.update([id])
    except KeyError:
        pass
delresults

previously_matched.loc[previously_matched.id.isin(delresults.keys())]
abbreviated_delegates.loc[abbreviated_delegates.id.isin(delresults.keys())]

,id,name,geboortejaar,sterfjaar,age_at_repr,colleges,functions,period,sg,was_gedeputeerde,p_interval,h_life,geboortedatum_p,overlijdensjaar


In [40]:
framed_gtlm = pd.DataFrame(moregentlemen)

In [41]:
framed_gtlm.rename(columns={0:'gentleobject'}, inplace=True)

In [42]:
framed_gtlm['vs'] = framed_gtlm.gentleobject.apply(lambda x: [e for e in x.variants['general']])
framed_gtlm['ref_id'] = framed_gtlm.gentleobject.apply(lambda x: x.heerid)
#framed_gtlm['uuid'] = framed_gtlm.gentleobject.apply(lambda x: x.get_uuid())
framed_gtlm['name'] = framed_gtlm.gentleobject.apply(lambda x: x.name)
framed_gtlm['found_in'] = day
framed_gtlm

,gentleobject,vs,ref_id,name,found_in
0,16901: Bout,"[Boat, Bout]",16901,Bout,1728
1,17355: de Drews,"[De Drewt, De Dreuws, De Drews, De Dreus, Drew...",17355,de Drews,1728
2,17821: Borre van Amerongen,"[Zaats van Ameuongen, Zaats van Amerongen, Zaa...",17821,Borre van Amerongen,1728
3,17077: van Borssele,"[Jan Borsselen, Van Borsselen, Van Borselen, B...",17077,van Borssele,1728
4,16465: van Isselmuden tot Zwollingerkamp,"[Van lielmuden, Van lselmuden, Vun lsselmuden,...",16465,van Isselmuden tot Zwollingerkamp,1728
...,...,...,...,...,...
84,17583: Pieck,"[sieckte, sieck]",17583,Pieck,1728
85,12480: Fockens,"[t'elokens, t'elckens]",12480,Fockens,1728
86,92: Buys,"[uytstel, uytten]",92,Buys,1728
87,14060: Boreel,"[vreemde, vreemt]",14060,Boreel,1728


In [43]:
def levenst_vals(x, txt):
    scores= [score_levenshtein_distance_ratio(i.form,txt) for i in x if len(i.form)>1]
    if max(scores) > 0.8:
        return max(scores)
    else:
        return np.nan


In [44]:
def delegates2spans(searchob):
    spans = searchob.matched_text.spans
    for span in spans:
        txt = searchob.matched_text.item[span.begin:span.end]
        msk = framed_gtlm.vs.apply(lambda x: levenst_vals(x, txt))
        mres = framed_gtlm.loc[msk==msk.max()]
        if len(mres)>0:
    #     if len(mres)>0:
    #         setattr(span, 'delegate_id', mres.uuid.iat[0])
    #         setattr(span, 'delegate_name', mres.name.iat[0])
        #print(kand, all_matched[kand])
            span.set_pattern(txt)
        if len(mres)>0:
            span.set_delegate(delegate_id=mres.ref_id.iat[0], delegate_name=mres.name.iat[0])


In [45]:
for T in searchobs:
    searchob = searchobs[T]
    #mo = MatchAndSpan(ob, junksweeper=junksweeper, previously_matched=previously_matched)
    delegates2spans(searchob)

In [46]:
merged_deps = pd.merge(left=framed_gtlm, right=abbreviated_delegates, left_on="ref_id", right_on="id", how="left")

In [47]:
serializable_df = merged_deps[['ref_id','geboortejaar', 'sterfjaar', 'colleges', 'functions',
       'period', 'sg', 'was_gedeputeerde', 'p_interval', 'h_life', 'vs','name_x', 'found_in',]]
serializable_df.rename(columns={'name_x':'name', 
                                'vs':'variants', 
                                "h_life": "hypothetical_life", 
                                "p_interval":"period_active", 
                                "sg": "active_stgen"},  inplace=True)


In [48]:
serializable_df.to_pickle('sheets/found_deputies.pickle')


In [49]:
for interval in ["hypothetical_life", "period_active"]:
    serializable_df[interval]=serializable_df[interval].apply(lambda x: [x.left, x.right])
serializable_df.to_excel('sheets/found_deputies.xlsx')

In [50]:
out=[]
for T in searchobs:
    ob = searchobs[T]
    out.append(ob.to_dict())

In [51]:
#with open('/Users/rikhoekstra/Downloads/1728_pres2.json', 'w') as fp:
json_year = json.dumps(obj=out)

In [55]:
with open('/Users/rikhoekstra/Downloads/1728_pres_20210420.json', 'w') as fp:
    json.dump(fp=fp, obj=out)

In [52]:
delegate_json = serializable_df[['ref_id', 'name', 'geboortejaar', 'sterfjaar', 'colleges',
       'functions', 'active_stgen', 'was_gedeputeerde',
       'period_active', 'hypothetical_life']].to_json(orient="records")

In [12]:
searchobs = runner.searchobs

In [203]:
searchobs = runner.searchobs
from IPython.display import HTML
t_out=[]
for T in searchobs:
    ob = searchobs[T].matched_text
    url = searchobs[T].make_url()
    ob.mapcolors()
    rest = ob.serialize()
    rest = f"""\n<tr><td><strong>{T}</strong></td><td>{rest}</td>"""
    if url:
        rest += f"""<td><a href='{url}'>link naar {T}-image</a></td>"""
    rest += "</tr>"
    t_out.append(rest)
#out.reverse()
outtable = "".join(t_out)
HTML(f"<table>{outtable}</table>")


session-1730-01-02-num-1,
session-1730-01-03-num-1,"PRA ESIDE, Den Heere van Wassenaer. PRAESENTIBUS , De Heeren een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt . Vanden Born, Bors vaa Waveren. Van Borsselen, Noey, van Hoorn. JTaats van a'm erongen , van Renswoude. Van Glnstra, met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte, van"
session-1730-01-04-num-1,"naer. PRAEZS ENTIBUS , De Heeren twee extraorainaris Gedeputeerden uyt de Provincie van Gelderlandt . Vanden Born, Bors wan Vaveren. Van Borsselen, Bout, Noey, van Hoorn. Van Renswoude. Van Glinstra , met een extraordinaris Gedeputeerde uyt de Pruvvincie van V 'rieslandt 'rieslandt 'rieslandt 'rieslandt"
session-1730-01-05-num-1,"ESIDE, Den Heere van WY assenaer. PRAESENTIBUS , De Heeren tw ee extraordinaris Gedeputeerden uyt de Provincie van Gelderlandt. Vanden Born, Bors van Waveren. Van Borsselen, Bout, Noey, van HoornTaats van Am erongen , van Renswoude. Van Glinstra, met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte , van"
session-1730-01-06-num-1,"PRA ESIDE, Den Heere van Wassenaer. PRAESENTIBUS , De Heeren van Y'ynbergen, met een extraordinuris Gedeputeerde uyt de Provincie van Gelderlandt. Vanden Boetzelaar, van Blocklandt, vanden Born, Bors van Haveren, vande Pol, de Raadt. Van Borsselen, Buut, Noey, van Hoorn. Taats van Am erongen , van Renswoude. Van Glinstra, met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolie, van"
session-1730-01-07-num-1,"0. PRA ESIDE, Den Heere van Wassenaer. PRAESEN TIEBUS , De Heeren van Wynbergen, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt . Van Blocklandi , vanden Born, vander Dussen, Berckhout, Bors uva Waveren, vande Poll , de Raadt. Van Borsselen, Bout, Nvey, van Hoorn. JTaats van Am erongen , van Renswoude. Van Glinstra, met een extraordinaris Gedeputcerde uytl de Provincie van Vrieslandt. Van Haarsolte, van"
session-1730-01-09-num-1,"0. Nihil actum est.Lunae den 9. Januarii1730. PRA ESIDE, Den Heere van Borsselen. PRAESENTIBUS , De Heeren JTorck, van Wynbergen , me t twee extraordinaris Gedeputeerden uyt de Provincie van Gelderlandt. Van Wassenaer, van den Boetzelaar, van Blocklandt, vanden Born, vander Duse. sen, Berckhout, Bors van Waveren, vande Poll , van Cattenborgh, de Raadt. Bout, Noey, vun Hoorn. Taats van Am erongen , van Renswoude. Van Glinstra , met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte, van met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte, van"
session-1730-01-10-num-1,"Martis den 10. Januarii1730, PRAESIDE Den Heere van Borselen . PRAESENTIBUS , De Heeren van Lynden, van Welderen, Torck, van W'ynbergen, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Wassenaer, vand en Boetzelaar, van Blocklandt, vanden Born, vander Dussen, Bors van Waveren, de Raadt. Bout, Noey, van Hoorn. JTaats van Am erongen , van Renswoude. Van Glinstra, met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte, van"
session-1730-01-11-num-1,"1730. P 1730. P 1730. P 1730. P 1730. P AZESIDE, Den Heere van Borsselen . PRAESEN TIEBUS , De Heeren van Lynden, Torck, van Woynbergen, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Hassenaer, van Blocklandt , vanden Born, Bors van WY'averen. Bout, Noey, van Hoorn. Taats van Am erongen , van Renswoude. Van Glinstra , met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte, van met een extraordinaris Gedeputeerde uyt de Provincie van Vrieslandt. Van Haarsolte, van"
session-1730-01-12-num-1,"PRA ESIDE, Den Heere van Borsselen. PRAESENTIBUS , De Heeren van Lynden , Torck, met een extraordinaris Gedeputeerde uyt de Provincie van Gelderlandt. Van Wassenaer, van Blocklandt, vanden Born, Bors van Waveren, Jongemaat. Bout, Noey, van Hoorn. Jaats van Am erongen , van Renswoude. Van Glinstra, met een ex

In [69]:
with open(f"/Users/rikhoekstra/Downloads/{day}_check.html", 'w') as flout:
    flout.write(f"<html><body><h1>results for {day}</h1>\n")
    flout.write(f"<table>{outtable}</table>")
    flout.write("</body></html>")

## To Elasticsearch

In [70]:
from republic.republic_keyword_matcher.elastic_search_helpers import bulk_upload
local_republic_es = Elasticsearch()
local_republic_es

NameError: name 'Elasticsearch' is not defined

In [ ]:
local_republic_es.indices.create(index='attendancelist')


In [ ]:
data_dict = out
bulk_upload(bulkdata=data_dict, index='attendancelist', doctype='attendancelist')